In [86]:
# !pip install --upgrade selenium==3.141.0

'3.141.0'

In [1]:
import pandas as pd
import numpy as np

import time

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By  # 3개는 selenium이 동작하면서 웹 화면이 로딩될때 까지 기다림을 지원
from selenium.webdriver.support.ui import WebDriverWait # 웹 드라이버 미리 설치
from selenium.webdriver.support import expected_conditions as EC       

In [72]:

driver = webdriver.Chrome('C:/Users/YOONSU/chromedriver.exe')
driver.get('https://www.google.co.kr/maps/place/Daepohang+Fish+Market/@38.1742891,128.6037221,17z/data=!4m7!3m6!1s0x5fd8bb1bfc51b231:0xda2236e4ed18c15b!8m2!3d38.1742891!4d128.6059161!9m1!1b1?hl=en')
driver.maximize_window()
time.sleep(5)

# sort to recent review 
wait = WebDriverWait(driver, 10)
menu_bt = wait.until(EC.element_to_be_clickable(
                       (By.XPATH, '//button[@data-value=\'Sort\']'))
                   )  
menu_bt.click()

recent_rating_bt = driver.find_elements_by_xpath(
                                     '//li[@role=\'menuitemradio\']')[0]
recent_rating_bt.click()


In [ ]:
'''
# specific attraction is differ from others, so we change html code for it.
driver = webdriver.Chrome('C:/Users/YOONSU/chromedriver.exe')
driver.get("https://www.google.co.kr/maps/place/Daemyung+Delpino+Golf+%26+Resort+%26+Hotel/data=!4m14!1m2!2m1!1z64247ZS864W4IOqzqO2UhOyVpOumrOyhsO2KuCDriIjsjbDrp6TsnqU!3m10!1s0x5fd8bd426d38d6fb:0x5cd1daca122d04bf!5m2!4m1!1i2!8m2!3d38.2120109!4d128.4940943!9m1!1b1!15sCinrjbjtlLzrhbgg6rOo7ZSE7JWk66as7KGw7Yq4IOuIiOyNsOunpOyepVouIizrjbjtlLzrhbgg6rOo7ZSEIOyVpCDrpqzsobDtirgg64iI7I2w66ekIOyepZIBDHJlc29ydF9ob3RlbJoBI0NoWkRTVWhOTUc5blMwVkpRMEZuU1VOWGNrNTZYMGwzRUFF?hl=en")
driver.maximize_window()
time.sleep(5)

# sort to recent review 
wait = WebDriverWait(driver, 10)
menu_bt = wait.until(EC.element_to_be_clickable(
                       (By.CSS_SELECTOR, '#QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf > div:nth-child(9) > button:nth-child(2) > div.IAbLGd'))
                   )  
menu_bt.click()

recent_rating_bt = driver.find_elements(By.XPATH,
                                     '//li[@role=\'menuitemradio\']')[1]
recent_rating_bt.click()
'''

In [73]:
# scroll code
# total n review (10 reviews per 1 page)
from tqdm import tqdm

# google map allows crawling until 93th scroll(maybe robots.txt disallow crawling)
page_num = 93
per = 10
pbar = tqdm(total=page_num*per)
with tqdm(total=page_num*per, position=0, leave=True) as pbar:
    for i in range(page_num):
        buffer = driver.find_elements_by_class_name('qjESne') # buffer location
        action = ActionChains(driver)

        n =len(buffer)
        action.move_to_element(buffer[n-1]).perform() # move to last buffer
        time.sleep(1)
        pbar.update(per)
    pbar.close()

100%|████████████████████████████████████████████████████████████████████████████████| 930/930 [03:05<00:00,  5.01it/s]


In [74]:
box_list = []
name_list = []
stars_list = []
review_list = []
date_list = []

boxes = driver.find_elements_by_class_name('jftiEf.fontBodyMedium') # review class


# more butten click
for box in boxes:
    try:
        more_btn = box.find_element_by_class_name('w8nwRe.kyuRq')
        more_btn.click()
        time.sleep(1)
    except:
        continue
# review collection
for box in boxes:

    IDs = box.find_elements_by_class_name('d4r55')

    Dates = box.find_elements_by_class_name('rsqaWe')

    Stars = box.find_elements_by_class_name('kvMYJc')

    reviews = box.find_elements_by_css_selector('span.wiI7pd')

    for ID, Date, Star, review in zip(IDs, Dates, Stars, reviews):
        name_list.append(ID.text)
        date_list.append(Date.text)
        stars_list.append(Star.get_attribute('aria-label'))
        review_list.append(review.text)


In [75]:
# review list to dataframe
review = pd.DataFrame(
    {'name': name_list,
     'rating': stars_list,
     'review': review_list,
     'date': date_list
    })
pd.set_option('display.max_rows', None)
print(len(review))
review.tail()

820


,name,rating,review,date
815,yang yang,1 star,"(Translated by Google) If you pay cash, they g...",11 months ago
816,happy be,5 stars,(Translated by Google) Kanghee's house is deli...,a year ago
817,엄정촌놈,4 stars,"(Translated by Google) It's cheap, it's delici...",9 months ago
818,모나리자,5 stars,(Translated by Google) I usually ate it from l...,2 years ago
819,Youn hee Cho,5 stars,(Translated by Google) D-dong Soseon Sashimi R...,2 years ago


In [76]:
eng_review = review.copy()

In [77]:
# remove korean name
eng_review['name2'] = eng_review['name'].str.findall(r'[ㄱ-ㅎㅏ-ㅣ가-힣]')
eng_review = eng_review[eng_review['name2'].str.len() == 0]

# remove korean review
eng_review['review2'] = eng_review['review'].str.findall(r'[ㄱ-ㅎㅏ-ㅣ가-힣]')
eng_review = eng_review[eng_review['review2'].str.len() == 0]

# remove name2 & review2 columns
eng_review.drop(['name2','review2'],axis=1, inplace=True)

# remove "(Translated by Google)" sentence
eng_review['review'] = eng_review['review'].str.replace('(Translated by Google) ','',regex=False)

# remove "Original" word
eng_review['review'] = eng_review['review'].str.replace('\(Original\)','',regex=False)

# remove not english sentence & special characters
eng_review['review'] = eng_review['review'].str.replace(r'[^A-Za-z0-9]',' ', regex=True)

# delete side blank and "\n"
eng_review['review'] = eng_review['review'].str.strip()

# replace with lowercase
eng_review['review'] = eng_review['review'].str.lower()

# remove "orginal" word
eng_review['review'] = eng_review['review'].str.replace('original','')

# remove unreviewed data
eng_review = eng_review[eng_review['review'].str.len()!=0]

print(len(eng_review))
eng_review.head(30)

63


,name,rating,review,date
0,Richard Jurasek,2 stars,very overpriced food cool to walk around but ...,a week ago
1,Nava K,3 stars,basically a market and seafood market perhap...,3 years ago
2,Nordiana Mat Salleh,5 stars,many variety of seafood fries over here and al...,3 years ago
3,Jinman Ha,5 stars,quite mordern and regacy market are be in same...,a year ago
4,Hong Gyu Park,5 stars,you must try if you want to eat something all ...,a year ago
5,Andy Koh Woon Chieh,3 stars,fresh seafood and quite empty during weekdays ...,3 years ago
6,jaja na,5 stars,so many seafood that gonna make you excited,3 years ago
7,Steve Choe,4 stars,a must place to visit great market place wit...,a year ago
8,AiMei Huang,4 stars,arrived there at night a must try fried seaf...,4 years ago
9,Karen Laurico,5 stars,it s one of the best fish market i ve visited ...,2 years ago


In [78]:
eng_review.tail(10)

,name,rating,review,date
56,mohd hussein hussein,5 stars,best,2 years ago
58,kmlai27 william,3 stars,ok,3 years ago
60,disen sakuoh,1 star,nice place to see,3 years ago
61,yi toby,5 stars,good,3 years ago
62,Firdaus Mohd,5 stars,nice,3 years ago
64,Seow Yuen Khen,4 stars,fresh seafood,3 years ago
65,Sabar Simatupang,5 stars,so wonderful,4 years ago
66,Dirk V,5 stars,interesting,2 years ago
109,พ่อจ๋าพาเที่ยว ตะลุยทั่วโลก,5 stars,of the fresh sea pointing to the point of dea...,2 years ago
164,ช่อผกา ปาณิกบุตร,3 stars,very fresh seafood affordable price crabs s...,2 years ago


In [79]:
#다 바꿔야함
eng_review['attraction'] = 0
eng_review['address'] = 0
eng_review['latitude'] = 38.1742891
eng_review['longitude'] = 128.6059161
#eng_review.to_csv("설악산권금성.csv", encoding="utf-8", index = None) #맥북 기준
eng_review.to_csv("속초/대포항수산시장.csv", encoding="utf-8", index = None) #window 기준

==================================================================================